# Automated Subtitle Downloader
## Download and Rename accordingly 
## Advanced with *BeautifulSoup*

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import time
from zipfile import ZipFile 
import os
from difflib import SequenceMatcher


In [9]:
def similarity(string1, string2):
    return SequenceMatcher(None, string1, string2).ratio()

def get_year():
    
    movie_year = input("Enter movie year: ")
    if len(movie_year) == 4 and movie_year.isnumeric():
        movie_year = int(movie_year)
        return movie_year
    else: print("Enter valid and numeric year"); get_year()
    

def get_language():
    lang_dict = {0: 'English', 1: 'Arabic', 2: 'Bengali', 3: 'Brazilian portuguese', 4: 'Bulgarian', 5: 'Chinese', 6: 'Croatian', 7: 'Czech', 8: 'Danish', 9: 'Hindi', 10: 'Farsi/persian', 
                 11: 'Finnish', 12: 'German', 13: 'Greek', 14: 'Hebrew', 15: 'Hungarian', 16: 'Indonesian', 17: 'Italian', 18: 'Japanese', 19: 'Korean', 20: 'Macedonian', 
                 21: 'Malay', 22: 'Norwegian', 23: 'Polish', 24: 'Portuguese', 25: 'Romanian', 26: 'Russian', 27: 'Serbian', 28: 'Slovenian', 29: 'Spanish', 30: 'Swedish', 
                 31: 'Thai', 32: 'Urdu', 33: 'Vietnamese'}
    print(lang_dict)
    print("Enter 0 for English language")
    
    lang_num = input("Chose and Enter subtitle language number from above: ")
    if (len(lang_num) == 1 or len(lang_num) == 2) and lang_num.isnumeric():
        
        lang_num = int(lang_num)
        try: 
            language = lang_dict[lang_num]
            return language
        except: print("Number not in Dict. Enter valid number"); get_language()
        
    else: print("Enter valid number"); get_language()
        
    

def enter_movie_details():
    
    language = get_language()
            
    movie_year = get_year()
        
    movie_name = input("Enter movie name: ")
    return movie_name, movie_year, language
 

def get_query_results(movie_name):
    query = movie_name.replace(" ", "%20")
    search_url = root_url + str("/search/") + str(query)
    page = requests.get(search_url)
    soup = bs(page.content, "lxml")
    results = soup.find_all("div", class_="media-body")
    return results


def get_subtitle_page_link(results, movie_name, movie_year):
    matched_year_dict = {}
    for i in range(len(results)):
        name = results[i].find("h3", class_="media-heading").text
        year = int(results[i].find_all("span", class_="movinfo-section")[0].text.strip().replace("year",""))
        
        href = results[i].find("a")['href']
        
        if year == movie_year:
            matched_year_dict[name] = href
    
    name_list = list(matched_year_dict.keys())
    print("name_list: ",name_list)
    prob_list = []
    for nam in name_list:
        nam = nam.lower()
        movie_nam = movie_name.lower()
        prob_list.append(similarity(nam, movie_nam))
    matched_href = matched_year_dict[name_list[prob_list.index(max(prob_list))]]
    sub_page_url = root_url + str(matched_href)
    return sub_page_url


def get_subtile_link(sub_page_url, language):
    page = requests.get(sub_page_url)
    soup = bs(page.content, "lxml")
    table = soup.find("table", { "class" : "table other-subs" })
    rating_dict = {}
    for row in table.findAll("tr"):
        cells = row.findAll("td")
        if len(cells) > 1:
            rating = cells[0].text.strip()
            lang = cells[1].text.strip()
            href = cells[2].find("a")['href']
            if lang == language:
                rating_dict[rating] = href

    best_href = rating_dict[max(rating_dict)]
    return best_href


def download_and_unzip_srt(sub_link):
    sub_link = sub_link.replace("subtitles","subtitle")
    sub_dwnld = "https://yifysubtitles.org" + sub_link + ".zip"
    r = requests.get(sub_dwnld, allow_redirects=True)
    sub_link = sub_link.split("/")[-1]
    file_name = dir_path_to_download + str(sub_link) + ".zip"
    open(file_name, 'wb').write(r.content)
    print('Downloaded') 
    with ZipFile(file_name, 'r') as zip: 
        #zip.printdir() # printing all the contents of the zip file 
        zip.extractall(dir_path_to_download)
        print('Extracted')


In [10]:
root_url = "https://yts-subs.com"
dir_path_to_download = "F:\\Movies_Tv_F\\Subtitles\\" 

movie_name, movie_year, language = enter_movie_details()
results = get_query_results(movie_name)
sub_page_url = get_subtitle_page_link(results, movie_name, movie_year)
best_href = get_subtile_link(sub_page_url, language)
download_and_unzip_srt(best_href)

{0: 'English', 1: 'Arabic', 2: 'Bengali', 3: 'Brazilian portuguese', 4: 'Bulgarian', 5: 'Chinese', 6: 'Croatian', 7: 'Czech', 8: 'Danish', 9: 'Hindi', 10: 'Farsi/persian', 11: 'Finnish', 12: 'German', 13: 'Greek', 14: 'Hebrew', 15: 'Hungarian', 16: 'Indonesian', 17: 'Italian', 18: 'Japanese', 19: 'Korean', 20: 'Macedonian', 21: 'Malay', 22: 'Norwegian', 23: 'Polish', 24: 'Portuguese', 25: 'Romanian', 26: 'Russian', 27: 'Serbian', 28: 'Slovenian', 29: 'Spanish', 30: 'Swedish', 31: 'Thai', 32: 'Urdu', 33: 'Vietnamese'}
Enter 0 for English language


Chose and Enter subtitle language number from above:  0
Enter movie year:  1979
Enter movie name:  mad max


name_list:  ['Mad Max']
Downloaded
Extracted


In [ ]:
# Hubie Halloween 2020

# END